# Synchronisation

Basically synchronization is needed so that there isn't a race between the coros, although this is supposed to be asynchronous programming, it could still need some order when excecuting. And for that we have the following, however it is important to have in mind that not most of the following work as intended, some aren't totally complete yet.

## Lock

A lock basically lets you put other functions in a wait state before it is released, meaning they are acquired by a function, and the other ones that wait for it, can't continue before it is released, this way all the functions can run one after another without problems. The order in which they run one after the other depends on the order they are defined.

Here we will show some examples, first we must connect with the board.

In [1]:
%serialconnect --port /dev/ttyUSB0 --baudrate 115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


Then first, `uasyncio` and `Lock` must be imported, as well as our function which will let us show how lock works and a killer function which defines a time in specific in which the processes will run.

In [2]:
import uasyncio as asyncio
from uasyncio.synchro import Lock

#async function defined to get the lock and after some time to release it
async def task(i,sleep, lock):
    while 1:
        print('Waiting lock in task ',i)
        await lock.acquire()
        print("Acquired lock in task ",i)
        await asyncio.sleep(sleep)
        print('Realeasing lock in task ',i)
        lock.release()

# killer function lets us run the asyncio example fot about 10 seconds, we pass loop so that we can stop it.
async def killer(loop):
    await asyncio.sleep(10)
    loop.stop()
    loop.close()

Then we initialize our `Lock` as follows, were as a parameter you can define a delay in ms., Remember it would be considered as a global instance.

In [3]:
lock = Lock()

And last, we add the desired tasks an run it.

In [4]:
loop=asyncio.get_event_loop()
loop.create_task(task(1, 0.5, lock))
loop.create_task(task(2, 1, lock))
loop.create_task(task(3, 1.5, lock))

loop.run_until_complete(killer(loop)) 

Waiting lock in task  1
Waiting lock in task  2
Waiting lock in task  3
Acquired lock in task  1
Realeasing lock in task  1
Waiting lock in task  1
Acquired lock in task  2
Realeasing lock in task  2
Waiting lock in task  2
Acquired lock in task  3
Realeasing lock in task  3
Waiting lock in task  3
Acquired lock in task  1
Realeasing lock in task  1
Waiting lock in task  1
Acquired lock in task  2
Realeasing lock in task  2
Waiting lock in task  2
Acquired lock in task  3
Realeasing lock in task  3
Waiting lock in task  3
Acquired lock in task  1
Realeasing lock in task  1
Waiting lock in task  1
Acquired lock in task  2
Realeasing lock in task  2
Waiting lock in task  2
Acquired lock in task  3
Realeasing lock in task  3
Waiting lock in task  3
Acquired lock in task  1
Realeasing lock in task  1
Waiting lock in task  1
Acquired lock in task  2


In [5]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

## Event

The event class is kinda like `Lock` one, the difference is that `Lock` puts all coros in some sort of order in which just one of them is run after the other, however event lets us stops one or more at the same time, and then let them continue at the same time. In order to use event, we should then import `asyn`.

In [6]:
import asyn
import uasyncio as asyncio
event = asyn.Event()

.

Then it is needed to define the coroutines which will be run, `task` is the one that runs depending on the `Event()`, whether if it is set or clear. 

In [7]:

async def task(i, event):
    while 1:
        await event
        if not event.is_set():
            print('waiting for event in task ',i)
        else:
            print('now running in task ',i)
        await asyncio.sleep(1)



Afterwards we define `eventset`, which sets and clears the event in periods so that `task` can run.

In [8]:
async def eventset(event):
    while 1:
        event.clear()
        print('event cleared')
        await asyncio.sleep(2)
        event.set()
        print('event set for a little')
        await asyncio.sleep(4)
        

async def killer(loop):
    await asyncio.sleep(10)
    loop.stop()
    loop.close()

In [9]:
loop = asyncio.get_event_loop()
loop.create_task(eventset(event))
loop.create_task(task(1,event))
loop.create_task(task(2,event))
loop.create_task(killer(loop))
loop.run_forever()

event cleared
event set for a little
now running in task  1
now running in task  2
now running in task  1
now running in task  2
now running in task  1
now running in task  2
now running in task  1
now running in task  2
event cleared
.event set for a little
now running in task  1
now running in task  2
now running in task  1
now running in task  2


In [10]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

## Barrier

This one basically lets a coro wait for another function to run before continuing. In this case some parameters must be specified like the amount of coros that will use it, the callback and the parameters of the callback. Lets run an example:


In [11]:
import asyn
import uasyncio as asyncio

#callback is defined
def callback(text):
    print(text)
    
#the first parameter corresponds to the amount of coros that will use the barrier
barrier = asyn.Barrier(3, callback, ('Synch',))

async def report(duration, barrier):
    for i in range(5):
        await asyncio.sleep(duration)
        print('{} '.format(i), end='')
        await barrier

async def killer():
    await asyncio.sleep(18)
        
loop = asyncio.get_event_loop()
loop.create_task(report(0.5, barrier))
loop.create_task(report(1, barrier))
loop.create_task(report(1.5, barrier))
loop.run_until_complete(killer())
loop.close()

0 0 0 Synch
1 1 1 Synch
2 2 2 Synch
3 3 3 Synch
.4 4 4 Synch
..

In [12]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

As you can see, with barrier, no other functions continues running until every of them is awaiting for the callback and don't continue until it is run.

## Semaphore

Semaphore, different from `Locks`, lets us put in queue coros, and in some way just a defined amount of them can keep running, just when the object semaphore is released. Lets run an example to understand it more.

First we must do the imports.

In [13]:
import uasyncio as asyncio
import asyn

Our functions are defined in order to see how semaphore works.

In [14]:
async def myWorker(i, duration, semaphore):
    while 1:
        await semaphore.acquire()
        print("Successfully acquired the semaphore", i)
        await asyncio.sleep(3)
        print("Releasing Semaphore", i)
        semaphore.release()
        await asyncio.sleep(duration)
    
async def killer(duration, loop):
        await asyncio.sleep(duration)
        loop.stop()
        loop.close()

In [15]:
sema=asyn.Semaphore(2)
loop = asyncio.get_event_loop()
loop.create_task(myWorker(1,2,sema))
loop.create_task(myWorker(2,3,sema))
loop.create_task(myWorker(3,1,sema))
loop.create_task(killer(15,loop))
loop.run_forever()

Successfully acquired the semaphore 1
Successfully acquired the semaphore 2
Releasing Semaphore 1
Successfully acquired the semaphore 3
Releasing Semaphore 2
Successfully acquired the semaphore 1
.Releasing Semaphore 3
Successfully acquired the semaphore 2
Releasing Semaphore 1
Successfully acquired the semaphore 3
Releasing Semaphore 2
Successfully acquired the semaphore 1
Releasing Semaphore 3
Successfully acquired the semaphore 3
.Releasing Semaphore 1
Successfully acquired the semaphore 2


In [16]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nMPY: soft reboot\r\nMicroPython v1.12 on 2019-12-20; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

## Queue

At last we have `Queue`, which lets us to pass values between coros. If the object Queue doesn't have anything inside, what is returned is None. First we must import needed modules and define our object Queue.

In [17]:
import uasyncio as asyncio
from uasyncio.queues import Queue
import urandom
q = Queue()

Then we must define our functions that will use Queue and the killer function.

In [18]:
async def slow_process():
    return urandom.randint(1,20)

async def producer(q):
    while True:
        result = await slow_process()  # somehow get some data
        await asyncio.sleep(2)
        await q.put(result) 
        

async def consumer(q):
    while True:
        result = await(q.get())  # Will pause if q is empty
        print('Result was {}'.format(result))


In [19]:
async def killer(duration, loop):
        await asyncio.sleep(duration)
        loop.stop()
        loop.close()

In [20]:
loop=asyncio.get_event_loop()
loop.create_task(producer(q))
loop.create_task(consumer(q))
loop.create_task(killer(10,loop))
loop.run_forever()

Result was 7
.Result was 9
Result was 4
.Result was 4


As you can see, when one result it put in queue, then the other can use it.